---
title: "Analytics Model"
format:
  html:
    embed-resources: true        
    toc: true
    code-fold: true
    math-method: mathjax         
    mathjax-url: "https://cdn.jsdelivr.net/npm/mathjax@2.7.9/MathJax.js?config=TeX-AMS-MML_HTMLorMML"
jupyter: python3
execute:
  working-directory: project
  echo: true
  warning: false
  error: true
---

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import adjusted_rand_score
eda = pd.read_parquet("data/eda.parquet")

In [ ]:
features = eda[['SALARY', 'MAX_YEARS_EXPERIENCE', 'MIN_YEARS_EXPERIENCE']].copy()

for col in ['MAX_YEARS_EXPERIENCE', 'MIN_YEARS_EXPERIENCE', 'SALARY']:
    features[col] = pd.to_numeric(features[col], errors='coerce')

features = features.dropna()

scaler = StandardScaler()
X = scaler.fit_transform(features)

kmeans = KMeans(n_clusters=4, random_state=688)
eda.loc[features.index, 'Cluster'] = kmeans.fit_predict(X)

true_labels = eda.loc[features.index, 'SOC_2021_4_NAME']
true_labels_encoded = LabelEncoder().fit_transform(true_labels)

ari = adjusted_rand_score(true_labels_encoded, eda.loc[features.index, 'Cluster'])

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import HTML
import numpy as np

# 1) Build the DataFrame
df_plot = features.copy()
df_plot['Cluster'] = eda.loc[features.index, 'Cluster']

# 1.1) Create a log salary column for better scaling
df_plot['Log_SALARY'] = np.log1p(df_plot['SALARY'])  # log(1+salary), avoid log(0)

# 2) Create an interactive Plotly Figure
fig = px.scatter(
    df_plot,
    x='Log_SALARY',  # 👈 use log salary instead of salary
    y='MAX_YEARS_EXPERIENCE',
    color='Cluster',
    title="KMeans Clustering by Log(Salary) and Max Years Experience",
    labels={
        'Log_SALARY': 'Log(Salary)',
        'MAX_YEARS_EXPERIENCE': 'Max Years Experience',
        'Cluster': 'Cluster'
    },
    width=800,
    height=600,
)

fig

<iframe
  src="/figures/analytics_plot1.html"
  width="100%"
  width="100%"
  style="border:none; height:90vh;
></iframe>

Here we have 4 cluster groups. Group 0, which represent as green have lower salary, mostly under 150k, and max years experience in 2-5 years, it is likely Likely junior to mid-level employees with moderate pay. Group 1 with orange, has medium to high salary, wide range from $100k–$500k and with narrow range ~3 years, they are suggests specialized or high-paying roles with short experience — possibly fast-track promotions or high-demand fields. cluster 2 are low salary and experience from 0-4 years, they are clearly entry level employee. cluster 3 has medium salary, mostly under 200k with higher experiences, like 6-13 eyars. They probably are senior professionals with more experience but not the highest salaries. 


In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import plotly.graph_objects as go

# Prepare features & target
features = eda[['MIN_YEARS_EXPERIENCE', 'MAX_YEARS_EXPERIENCE']].apply(pd.to_numeric, errors='coerce')
features = features.dropna()
X = features
y = eda.loc[X.index, 'SALARY']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=688)

# Fit model & predict
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Metrics (optional, but handy)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MSE: {mse:.2f}, R²: {r2:.3f}")

# Define min/max for the identity line
min_val = y_test.min()
max_val = y_test.max()

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Apply log transform
y_test_log = np.log1p(y_test)
y_pred_log = np.log1p(y_pred)

# Plot
fig = go.Figure([
    go.Scatter(
        x=y_test_log,
        y=y_pred_log,
        mode='markers',
        marker=dict(color='skyblue', opacity=0.6),
        name='Predicted vs Actual (Log Scale)'
    ),
    go.Scatter(
        x=[min(y_test_log), max(y_test_log)],
        y=[min(y_test_log), max(y_test_log)],
        mode='lines',
        line=dict(color='red', dash='dash'),
        name='Ideal Fit'
    )
])

fig.update_layout(
    autosize=True,
    height=400,
    title="Predicted vs Actual Salary (Log Scale)",
    xaxis_title="Actual Log(Salary)",
    yaxis_title="Predicted Log(Salary)",
    margin=dict(l=20, r=20, t=50, b=20)
)

fig.write_html(
    "figures/analytics_plot2.html",
    full_html=False,
    include_plotlyjs="cdn",
    config={"responsive": True}
)

fig.show()

<iframe
  src="/figures/analytics_plot2.html"
  width="100%"
  style="border:none; height:90vh;
></iframe>

This plot shows the Actual vs. Predicted Salary using a multiple linear regression model. The blue dots represent individual predictions, and the red dashed line is the ideal line where predicted = actual. Since most points lie very close to the red line, it means your model predicts salary very accurately, with minimal error and strong linear fit — likely reflected in a high R² score near 1.0.
